# Goal: Crack Password
We are going to try and determine a hidden password.

# Choosing a fitness Function
The evaluation function is the first step to create a genetic algorithm. It’s the function that estimates the success of our specimen. The simplest solution can be shown as:

```
fitness score = (number of char correct) / (total number of char)
```


In [1]:
def fitness(password, test_word): 
  if (len(test_word) != len(password)):
    print('Not compatible')
    return
  else: 
    score = sum([1 for i, j in zip(password, test_word) if i == j])
    return score * 100 / len(password)

In [2]:
test_pass = 'banana' 
test_word = 'lawinl'

In [3]:
fitness(test_pass, test_word)

33.333333333333336

# Creating Individuals
So now we know how to evaluate our individuals; but how do we define them? This part is really tricky: the goal is to know what are the unalterable characteristics and what is variable.

The comparison with genetics is here really helpful. Indeed, the DNA is composed of genes, and each of those genes comes through different alleles (different versions of this gene). Genetic algorithms retain this concept of population’s DNA.

In our case, our individuals are going to be words (obviously of equal length with the password). Each letter is a gene and the value of the letter is the allele. In the word “banana”: ‘b’ is the allele of the first letter.

What is the point of this creation?

We know that each of our individuals is keeping the good shape (a word with the correct size)
Our population can cover every possibility (every word possible with this size).
Out genetic algorithm can then explore all possible combinations.

# Creating our first population
Now, we know what are the characteristics of our individuals and how we can evaluate their performance. We can now start the “evolution” step of our genetic algorithm.

The main idea to keep in mind when we create the first population is that we must not point the population towards a solution that seems good. We must make the population as wide as possible and make it cover as many possibilities as possible. The perfect first population of a genetic algorithm should cover every existing allele.

So in our case, we are just going to create words only composed of random letters.

In [4]:
import random 

def generateAWord(length):
  result = ""
  for i in range(length):
    letter = chr(97 + int(26 * random.random()))
    result += letter
  return result  

def generateFirstPopulation(size, password):
  population = []
  for i in range(size):
    population.append(generateAWord(len(password)))
  return population

In [5]:
generateAWord(10)

'ggtbesoqce'

In [6]:
generateFirstPopulation(10, 'banana')

['tchlds',
 'svpeni',
 'zkkivj',
 'xxpkgz',
 'rjgojt',
 'bcuvdf',
 'kmvcwp',
 'csbzck',
 'jedips',
 'gddkeo']

# From one generation to the next
Given a generation, in order to create the next one, we have 2 things to do. 
1. First we select a specific part of our current generation. 
2. Then the genetic algorithm combines those breeders in order to create the next batch.

## Breeders selection
They are lots of way to do this but you must keep in mind two ideas: the goals are to select the best solutions of the previous generation and not to completely put aside the others. The hazard is: if you select only the good solutions at the beginning of the genetic algorithm you are going to converge really quickly towards a local minimum and not towards the best solution possible.

My solution to do that is to select on the one hand the _N_ better specimen (in our code, _N_ = best_sample) and on the other hand to select _M_ random individuals without distinction of fitness (_M_ = lucky_few).

In [7]:
import operator 

def computeRankedPopulation(population, password):
  populationRanked = {}
  for individual in population:
    populationRanked[individual] = fitness(password, individual)
  return sorted(populationRanked.items(), key=operator.itemgetter(1), reverse=True)

def selectFromPopulation(population, best_sample, lucky_few):
  nextGeneration = []
  for i in range(best_sample):
    nextGeneration.append(population[i][0])
  for i in range(lucky_few):
    nextGeneration.append(random.choice(population)[0])
  return random.shuffle(nextGeneration)